### Analysis description
Create nifti mask files from ROIs drawing based on Fullscreen retinotopy

In [ ]:
# Imports
import os
import nibabel as nb
import ipdb
import warnings
import cortex
from pathlib import Path
warnings.filterwarnings('ignore')

# Define parameters
subjects = ['sub-001','sub-002','sub-003','sub-004',
            'sub-005','sub-006','sub-007','sub-008']
subjects_plot = ['sub-001','sub-002','sub-003','sub-004',
                 'sub-005','sub-006','sub-007','sub-008']
gaze_tasks = ['FullScreen','GazeCenter','GazeLeft','GazeRight']
attend_tasks = ['AttendFix','AttendBar']
rois = ['V1','V2','V3','V3AB','hMT+','LO','VO','iIPS','sIPS','iPCS','sPCS','mPCS']

# Define folders
base_dir = '/home/mszinte/disks/meso_S/data/gaze_prf'
bids_dir = "{}".format(base_dir)
pp_dir = "{}/derivatives/pp_data".format(base_dir)
pycortex_dir = "{}/cortex".format(pp_dir)
cortical_mask = 'cortical'
xfm_name = 'identity.fmriprep'

In [ ]:
# Write pycortex config file
pycortex_db = "{}/db/".format(pycortex_dir)
pycortex_cm = "{}/colormaps/".format(pycortex_dir)
pycortex_config_file  = cortex.options.usercfg
pycortex_config_file_new = pycortex_config_file[:-4] + '_new.cfg'
pycortex_config_file_old = pycortex_config_file[:-4] + '_old.cfg'

Path(pycortex_config_file_new).touch()
with open(pycortex_config_file, 'r') as fileIn:
    with open(pycortex_config_file_new, 'w') as fileOut:
        for line in fileIn:
            if 'filestore' in line:
                newline = 'filestore=' + pycortex_db
                fileOut.write(newline)
                newline = '\n'
            elif 'colormaps' in line:
                newline = 'colormaps=' + pycortex_cm
                fileOut.write(newline)
                newline = '\n'
            else:
                newline = line
            fileOut.write(newline)
os.rename(pycortex_config_file,pycortex_config_file_old)
os.rename(pycortex_config_file_new, pycortex_config_file)

In [ ]:
for subject in subjects:
    
    print(subject)
    
    # Define and create folder
    mask_dir = '{}/{}/masks'.format(pp_dir, subject)
    try: os.makedirs(mask_dir) 
    except: pass

    # Create ROI masks
    # ----------------
    ref_file = "{}/db/{}/transforms/{}/reference.nii.gz".format(pycortex_dir, subject, xfm_name)
    ref_img = nb.load(ref_file)

    for roi in rois:
        roi_mask_file_L = "{}/{}_{}_L.nii.gz".format(mask_dir, roi, cortical_mask)

        if os.path.isfile(roi_mask_file_L) == 0:
            print('creating {} {} mask'.format(roi, cortical_mask))
            roi_mask = cortex.utils.get_roi_masks(subject=subject, xfmname=xfm_name, gm_sampler=cortical_mask, roi_list=roi, return_dict=True, split_lr=True)
            for hemi in ['L','R']:
                roi_mask_file = "{}/{}_{}_{}.nii.gz".format(mask_dir, roi, cortical_mask, hemi)
                roi_mask_img = nb.Nifti1Image(dataobj=roi_mask['{}_{}'.format(roi, hemi)].transpose((2,1,0)), affine=ref_img.affine, header=ref_img.header)
                roi_mask_img.to_filename(roi_mask_file)
        else:
            print('{} {} masks: already exist'.format(roi, cortical_mask))
            continue